Training, Validation and testing dataset were labeled by batch manually

In [6]:
import sys
sys.path.append('/host/d/Github/')
import os
import numpy as np
import pandas as pd
import nibabel as nb
import Diffusion_denoising_thin_slice.functions_collection as ff

In [7]:
patient_sheet = ff.find_all_target_files(['*'],os.path.join('/host/d/Data/low_dose_CT/nii_imgs'))
print('total patient num: ', len(patient_sheet))

total patient num:  10


In [8]:
# shuffle these patients
np.random.seed(42)
shuffled_indices = np.random.permutation(len(patient_sheet))
patient_sheet = [patient_sheet[i] for i in shuffled_indices]

In [9]:
# 
results = []
for i in range(0, len(patient_sheet)):
    patient_id = os.path.basename(patient_sheet[i])
    # first 6 are train, seven and eight are val, last three are test
    if i < 6:
        batch = 'train'
    elif i == 6:
        batch = 'val'
    else:
        batch = 'test'
    results.append([batch, patient_id])
df = pd.DataFrame(results, columns=['batch','Patient_ID'])
save_path = os.path.join('/host/d/Data/low_dose_CT/Patient_lists','mayo_low_dose_CT_batched.xlsx')
df.to_excel(save_path, index=False)

# build list for simulations

In [11]:
patient_sheet = pd.read_excel(os.path.join('/host/d/Data/low_dose_CT/Patient_lists/mayo_low_dose_CT_batched.xlsx'),dtype={'Patient_ID': str, 'Patient_subID': str})
noise_types = ['gaussian','poisson']
simulation_num = 2

data_path = '/host/d/Data/low_dose_CT/'

for noise_type in noise_types:
    results = []
    for i in range(0, len(patient_sheet)):
        patient_id = patient_sheet['Patient_ID'][i]
        batch = patient_sheet['batch'][i]
        print(f"Processing patient {patient_id} in batch {batch} with noise type {noise_type}")


        for n in range(0,simulation_num):
            n_type = 'gaussian' if noise_type == 'gaussian' else 'possion'
            simulation_file_odd = os.path.join(data_path,'simulation_v1', patient_id, n_type + '_random_' + str(n), 'recon_odd.nii.gz')
            simulation_file_even = os.path.join(data_path,'simulation_v1', patient_id, n_type + '_random_' + str(n), 'recon_even.nii.gz')

            ground_truth_file = os.path.join(data_path,'nii_imgs', patient_id, 'img.nii.gz')

            results.append([batch,patient_id, n, simulation_file_odd, simulation_file_even, ground_truth_file])

        df = pd.DataFrame(results, columns=['batch','Patient_ID', 'random_num', 'simulation_file_odd', 'simulation_file_even', 'ground_truth_file'])
        df.to_excel(os.path.join('/host/d/Data/low_dose_CT/Patient_lists', 'mayo_low_dose_CT_' + noise_type + '_simulation_v1.xlsx'), index=False)



Processing patient L333 in batch train with noise type gaussian
Processing patient L096 in batch train with noise type gaussian
Processing patient L286 in batch train with noise type gaussian
Processing patient L067 in batch train with noise type gaussian
Processing patient L310 in batch train with noise type gaussian
Processing patient L109 in batch train with noise type gaussian
Processing patient L506 in batch val with noise type gaussian
Processing patient L192 in batch test with noise type gaussian
Processing patient L143 in batch test with noise type gaussian
Processing patient L291 in batch test with noise type gaussian
Processing patient L333 in batch train with noise type poisson
Processing patient L096 in batch train with noise type poisson
Processing patient L286 in batch train with noise type poisson
Processing patient L067 in batch train with noise type poisson
Processing patient L310 in batch train with noise type poisson
Processing patient L109 in batch train with noise 

# build list for distilled model

In [3]:
patient_sheet = pd.read_excel(os.path.join('/mnt/camca_NAS/denoising/Patient_lists/fixedCT_static_shuffled_batched.xlsx'),dtype={'Patient_ID': str, 'Patient_subID': str})
simulation_num = 1

link = 'local' # 'local' or 'NAS'
if link == 'local':
    data_path1 = '/workspace/Documents/Data/denoising'
# elif link == 'NAS':
#     data_path1 = '/mnt/camca_NAS/denoising/Data'

results = []
for i in range(0, len(patient_sheet)):
    patient_id = patient_sheet['Patient_ID'][i]
    patient_subid = patient_sheet['Patient_subID'][i]
    batch = patient_sheet['batch'][i]
    print(f"Processing patient {patient_id} {patient_subid}...")


    for n in range(0,simulation_num):
        simulation_file = os.path.join(data_path1,'pred_images/',patient_id, patient_subid, 'random_'+str(n), 'condition_img.nii.gz')

        ground_truth_file = os.path.join(data_path1,'pred_images/',patient_id,patient_subid,'random_'+str(n), 'pred_img_scans20.nii.gz')

        results.append([batch,patient_id, patient_subid, n,simulation_file, ground_truth_file])#, simulation_img.shape])

        df = pd.DataFrame(results, columns=['batch','Patient_ID', 'Patient_subID', 'random_num', 'noise_file', 'ground_truth_file'])#, 'image_shape'])
        df.to_excel(os.path.join('/mnt/camca_NAS/denoising/Patient_lists/fixedCT_static_distilled_model_train_test_' + link +'.xlsx'), index=False)

Processing patient 00139437 0000258390...
Processing patient 00019599 0000029506...
Processing patient 00148611 0000455363...
Processing patient 00173704 0000455358...
Processing patient 00030597 0000455473...
Processing patient 00214862 0000455530...
Processing patient 00004038 0000455420...
Processing patient 00214775 0000455224...
Processing patient 00214865 0000455504...
Processing patient 00214905 0000455573...
Processing patient 00214815 0000455367...
Processing patient 00214925 0000455633...
Processing patient 00214918 0000455613...
Processing patient 00214919 0000455609...
Processing patient 00214864 0000455532...
Processing patient 00010431 0000019300...
Processing patient 00214935 0000455654...
Processing patient 00010536 0000455747...
Processing patient 00058992 0000455611...
Processing patient 00010795 0000455825...
Processing patient 00171178 0000455242...
Processing patient 00093479 0000455522...
Processing patient 00008527 0000455795...
Processing patient 00154227 000045